1. Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных 
результатах (поле mask в results.pkl). 
Для унификации предлагаю:

* взять в тренировочный набор турниры с dateStart из 2019 года; 
* в тестовый — турниры с dateStart из 2020 года.

In [1]:
import pickle
import pandas as pd
import numpy as np
import re
from itertools import chain
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression,LinearRegression
from scipy.special import logit, expit
from copy import deepcopy
from scipy.stats import spearmanr, kendalltau
pd.options.mode.chained_assignment = None

In [2]:
with open('players.pkl','rb') as p, open('results.pkl','rb') as r, open('tournaments.pkl','rb') as t:
    players = pickle.load(p)
    results = pickle.load(r)
    tournaments = pickle.load(t)

In [3]:
tournaments = pd.DataFrame.from_dict(tournaments,orient='index').set_index('id')
tournaments['year'] = tournaments['dateStart'].apply(lambda x: x[:4])

tournaments_train = tournaments[tournaments['year'] == '2019']
tournaments_test = tournaments[tournaments['year'] == '2020']
tour_train_ids = tournaments_train.index.to_list()
tour_test_ids = tournaments_test.index.to_list()

In [4]:
players = pd.DataFrame.from_dict(players, orient='index').set_index('id')
players.replace(to_replace=[None], value='', inplace=True)
players.fillna('')
players['name'] = players['name']+' ' + players['patronymic'] + ' ' + players['surname']
players = players[['name']]
players.head(10)
players.head(10)

,name
id,
1,Алексей Абабилов
10,Игорь Абалов
11,Наталья Юрьевна Абалымова
12,Артур Евгеньевич Абальян
13,Эрик Евгеньевич Абальян
14,Василий Абанин
15,Олег Игоревич Абарников
16,Азер Абасали оглы Абасалиев
17,А. В. Абасев


In [5]:
tour_lst = []
team_lst = []
player_lst = []
mask_lst = []
position_lst = []

for tour_id in results:
    if tour_id in tour_train_ids or tour_id in tour_test_ids:
        for team in results[tour_id]:
            mask = team.get('mask')
            if 'mask' in team:
                if  not mask or re.findall('[^01]', mask):
                    continue 
                for member in team['teamMembers']:
                    tour_lst.append(tour_id)
                    team_lst.append(team['team']['id'])
                    player_lst.append(member['player']['id'])
                    mask_lst.append(team['mask'])
                    position_lst.append(team['position'])
                    
data = pd.DataFrame({'tournament': tour_lst, 'team': team_lst, 'player': player_lst,'mask': mask_lst, 
                     'position': position_lst})

data['number_of_questions'] = data['mask'].str.len()
data = data[data['number_of_questions']==36]
data

,tournament,team,player,mask,position,number_of_questions
0,4772,45556,6212,111111111011111110111111111100010010,1.0,36
1,4772,45556,18332,111111111011111110111111111100010010,1.0,36
2,4772,45556,18036,111111111011111110111111111100010010,1.0,36
3,4772,45556,22799,111111111011111110111111111100010010,1.0,36
4,4772,45556,15456,111111111011111110111111111100010010,1.0,36
...,...,...,...,...,...,...
481665,6410,60732,179291,000000000000000000000011000000000000,4.0,36
481666,6410,60732,161570,000000000000000000000011000000000000,4.0,36
481667,6410,60732,197302,000000000000000000000011000000000000,4.0,36
481668,6410,63157,217110,000000000000000000000000000000000000,5.0,36


In [6]:
questions = []
for n in data['number_of_questions']:
    questions.extend(np.arange(1, n + 1))
    
data_questions_separated = pd.DataFrame({
    'tournament': np.repeat(data['tournament'], data['number_of_questions']),
    'position': np.repeat(data['position'], data['number_of_questions']),
    'team': np.repeat(data['team'], data['number_of_questions']),
    'player': np.repeat(data['player'], data['number_of_questions']),
    'question': questions,
    'answer': list(chain.from_iterable(data['mask']))
})
data_questions_separated['question_id'] = data_questions_separated['tournament'].astype(str) + '_' \
+ data_questions_separated['question'].astype(str)

data_questions_separated = data_questions_separated[data_questions_separated['answer'].isin(['0', '1'])]
data_questions_separated.loc[:, 'answer'] = data_questions_separated['answer'].astype(int, copy=False)

data_train = data_questions_separated[data_questions_separated['tournament'].isin(tour_train_ids)]
data_test = data_questions_separated[data_questions_separated['tournament'].isin(tour_test_ids)]


In [7]:
data_train.head()

,tournament,position,team,player,question,answer,question_id
0,4772,1.0,45556,6212,1,1,4772_1
0,4772,1.0,45556,6212,2,1,4772_2
0,4772,1.0,45556,6212,3,1,4772_3
0,4772,1.0,45556,6212,4,1,4772_4
0,4772,1.0,45556,6212,5,1,4772_5


2. 
Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. 
Замечания и подсказки:
* повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;

* в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; 

* скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;

* для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.

Обучим логистическую регрессию на one-hot матрице из игроков и вопросов. Целевая переменная - правильный ли был ответ на вопрос

In [8]:
encoder = OneHotEncoder(handle_unknown='ignore')
X_train = encoder.fit_transform(data_train[['player', 'question_id']])
X_test = encoder.transform(data_test[['player', 'question_id']])
y_train = data_train['answer']
y_test = data_test['answer']

In [9]:
baseline = LogisticRegression(n_jobs=-1)
baseline.fit(X_train, y_train)
predictions = baseline.predict_proba(X_test)[:, 1]

In [10]:
weights = baseline.coef_[0]
players_unique = np.unique(data_train['player'])
questions_unique = np.unique(data_train['question_id'])
                    
rating = pd.DataFrame({'id': players_unique,
                       'rating': weights[:len(players_unique)]})

rating =  rating.join(players, on=["id"], how="inner")
rating.sort_values(by='rating', ascending=False).head(25)

,id,rating,name
3456,27822,3.674244,Михаил Владимирович Савченков
3397,27403,3.579448,Максим Михайлович Руссо
3242,26089,3.565995,Ирина Сергеевна Прокофьева
530,4270,3.555879,Александра Владимировна Брутер
3750,30270,3.475793,Сергей Леонидович Спешков
3563,28751,3.365796,Иван Николаевич Семушин
3781,30475,3.301776,Владимир Владимирович Степанов
1841,14786,3.290938,Николай Александрович Коврижных
4284,34846,3.266914,Антон Анатольевич Чернин
3732,30152,3.250177,Артём Сергеевич Сорожкин


3. Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты 
наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах;
да и не нужны эти предсказания сами по себе. Поэтому:

* предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, 
в виде ранжирования команд;
* в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла 
(их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, 
усреднённые по тестовому множеству турниров.

Пусть вероятность ответа $i$ игрока в текущем турнире $p_{player_i}$, тогда общая сила команды  $p_{team} = 1 - \prod\limits_i (1 - p_{player_i})$ 

In [11]:
def correlations(data, predictions):
    spearman_corr = []
    kendall_corr = []
    pred_strength = data[['tournament', 'team']]
    pred_strength['pred_negative'] = 1 - predictions
    pred_strength = pred_strength.groupby(['tournament', 'team']).prod().reset_index()
    pred_strength['pred_position'] = pred_strength.groupby('tournament')['pred_negative'].rank('dense')
    true_strength = data[['tournament', 'team', 'position']].drop_duplicates()
    true_and_pred = pd.merge(pred_strength, true_strength, on=['tournament', 'team'])

    for tournament in true_and_pred['tournament'].unique():
        cur_tournament = true_and_pred[true_and_pred['tournament'] == tournament]
        if len(cur_tournament) > 1:
            spearman_corr.append(spearmanr(cur_tournament['position'], cur_tournament['pred_position'])[0])
            kendall_corr.append(kendalltau(cur_tournament['position'], cur_tournament['pred_position'])[0])
    return np.mean(spearman_corr), np.mean(kendall_corr)

In [12]:
spearman_corr, kendall_corr = correlations(data_test, predictions)
print('Базовая модель:')
print('Корреляция Спирмена: ', spearman_corr)
print('Корреляция Кендалла: ', kendall_corr) 

Базовая модель:
Корреляция Спирмена:  0.737081766567115
Корреляция Кендалла:  0.5799119322259015


### 4. EM-схема

Будем считать, что если команда не ответила на вопрос, то никто из команды не знал ответа на вопрос, и что команда ответила на вопрос, если хотя бы один игрок команды ответил на вопрос. Введем скрытую переменную $z_{ij}$: вероятность ответа игрока $i$ на вопрос $j$ при условии параметров модели и ответов. В качестве начального приближения возьмем предсказания бейзлайн модели.

E-шаг: фиксируем веса игроков и вопросов, вычисляем ожидание скрытой переменной. 

$$E(z_{ij}|\theta_n,y)=\begin{cases} 
\frac{p(z_{ij}|\theta_n)}{1-\prod\limits_i(1-p(z_{ij}|\theta_n)}, &\text{если $y = 1$}\\
0, &\text{если $y = 0$}
\end{cases}$$
М-шаг: фиксируем $z_{ij}$ и  обучаем логистическую регрессию с "мягкими" (вероятностными) метками. <br>


In [13]:
train_em = deepcopy(data_train)
train_em['players_strength'] = baseline.predict_proba(X_train)[:, 1]
model_em = LinearRegression()

def E_step(data):
    data['pred_negative'] = 1 - data['players_strength']
    teams_em = 1 - data.groupby(['tournament', 'team', 'question'])['pred_negative'].prod()
    data = data.merge(teams_em.rename('team_strength'), left_on=['tournament', 'team', 'question'], right_index=True)
    data['z'] = data['players_strength'] / data['team_strength']
    data['z'] = np.where(y_train == 0, 0, data['z'])
    data['z'] = np.clip(data['z'], 1e-6, 1 - 1e-6)
    return data
    
def M_step(data):
    model_em.fit(X_train, logit(data['z']))
    data['players_strength'] = expit(model_em.predict(X_train))
    data = data.drop('team_strength',1)
    return data

for step in range(1,4):
    print('Шаг', step)
    train_em = E_step(train_em)
    train_em = M_step(train_em)
   
    test_predictions = expit(model_em.predict(X_test))
    spearman_corr, kendall_corr = correlations(data_test,test_predictions)
    print('Корреляция Спирмена: ', spearman_corr)
    print('Корреляция Кенделла: ', kendall_corr)

Шаг 1
Корреляция Спирмена:  0.761783893051727
Корреляция Кенделла:  0.6044638381004592
Шаг 2
Корреляция Спирмена:  0.7687437377368573
Корреляция Кенделла:  0.6120997621196859
Шаг 3
Корреляция Спирмена:  0.7701569518556167
Корреляция Кенделла:  0.6132694452627618


Видно, что корреляции медленно, но растут. Построим обновленный рейтинг игроков.

In [21]:
weights_new = model_em.coef_
rating_new = pd.DataFrame({'id': players_unique,
                          'rating': weights_new[:len(players_unique)]})

rating_new = rating_new.join(players, on=["id"], how="inner")
rating_new.sort_values(by='rating', ascending=False).head(25)

,id,rating,name
45127,212663,12.344232,Майя Александровна Губина
4529,36844,11.164822,Павел Константинович Щербина
3333,26798,9.857505,Анна Григорьевна Резникова
48370,216994,9.597205,Артём Георгиевич Баранов
34853,201102,9.242593,Софья Павловна Молчанова
19647,168352,9.013553,Михаил Сергеевич Григорьев
2712,21661,8.676808,Артём Валерьевич Москаленко
39452,206275,8.243017,Руслан Васильевич Гайфутдинов
35983,202410,8.229609,Валентина Подюкова
13783,136300,8.227322,Александра Петровна Буйная


5. А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).


Посортируем турниры по сложности вопросов (ее возьмем из коэффициентов ЕМ-модели)

In [14]:
question_weights = dict(zip(encoder.categories_[1], model_em.coef_[:encoder.categories_[1].shape[0]]))
rating_tournaments = data_train[['tournament', 'question', 'question_id']].drop_duplicates()
rating_tournaments ['question_difficulty'] = rating_tournaments['question_id'].map(question_weights)
rating_tournaments  = rating_tournaments .groupby('tournament')['question_difficulty'].mean().reset_index()
rating_tournaments  = rating_tournaments .merge(tournaments[['name']], left_on='tournament', right_index=True)
rating_tournaments  = rating_tournaments .sort_values(by='question_difficulty', ascending=False)

In [15]:
rating_tournaments.head(20)

,tournament,question_difficulty,name
130,5563,3.877033,Линч
62,5395,3.690588,Синхрон Первенства Сибири
129,5558,3.612628,ТРИОтлон-2
131,5568,3.564813,Голова профессора Доуэля
105,5507,3.556628,Кубок главы города Иваново
112,5514,3.510733,Кубок Космонавтики
104,5504,3.503743,Весенний Синхронный Умлаут
69,5411,3.488326,Серия Premier. Мартовские иды
102,5502,3.475904,Чемпионат Выборга
66,5402,3.464237,Триптих. Осень


In [16]:
rating_tournaments.tail(20)

,tournament,question_difficulty,name
350,6138,1.340618,Синхронный кубок МГУ. Вторая пара
352,6144,1.288920,Из Минска с любовью. Этап 3
328,6068,1.226213,Чемпионат Минска. Лига Б. Тур второй
299,5963,1.221253,Асинхрон по South Park
347,6122,1.207075,Гран-при Бауманки. 2 этап. Кубок весей
339,6103,1.199121,Українська ліга. Етап 2
322,6024,1.193069,Gamer - 1
280,5900,1.191295,С берегов Оби
353,6146,1.176380,Избранное Осеннего Кубка Барнаула
346,6120,1.107752,Открытый кубок МВУТ


В конец рейтинга попали турниры ВУЗов, школ. В начало рейтинга попали более значимые турниры, рейтинг соответствует интуиции